In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt


# Load the dataset
df = pd.read_csv('Processed Data/Instances Imputed.csv', delimiter='\t')

# Drop non-feature columns
X = df.drop(columns=['Stress Level', 'Date/Time'])
y = df['Stress Level']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize the features
scaler = MinMaxScaler()#StandardScaler()
X_scaled = scaler.fit_transform(X)

split_index_train = int(0.8 * len(X_scaled))
split_index_val = int(0.9 * len(X_scaled))

X_train, X_val, X_test = X_scaled[:split_index_train], X_scaled[split_index_train:split_index_val],X_scaled[split_index_val:]
y_train, y_val, y_test = y_encoded[:split_index_train], y_encoded[split_index_train:split_index_val],y_encoded[split_index_val:]


In [25]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, Add, Flatten, Lambda
from keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.metrics import precision_score, recall_score, f1_score

# Transformer Block
def transformer_block(inputs, head_size, num_heads, ff_dim, dropout=0):
    attn_output = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    attn_output = Dropout(dropout)(attn_output)
    out1 = Add()([inputs, attn_output])
    out1 = LayerNormalization(epsilon=1e-6)(out1)
    
    ffn_output = Dense(ff_dim, activation="relu")(out1)
    ffn_output = Dropout(dropout)(ffn_output)
    
    ffn_output = Dense(inputs.shape[-1], activation="relu")(ffn_output)  # Ensure same shape as input
    out2 = Add()([out1, ffn_output])
    return LayerNormalization(epsilon=1e-6)(out2)

# Build the model
input_shape = (13,)
inputs = Input(shape=input_shape)

# Expand dimensions for multi-head attention
x = Dense(64)(inputs)
x = Lambda(lambda x: tf.expand_dims(x, axis=1))(x)  # Add a sequence dimension

# Transformer layers
x = transformer_block(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)
x = transformer_block(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)

# Flatten and add classification head
x = Flatten()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(3, activation="softmax")(x)

# Compile the model
model = Model(inputs, outputs)

# Calculate class weights
class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)))

# Compile the model without specifying accuracy
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy')

# Model summary
model.summary()

# Custom Metrics Callback
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_predict = np.argmax(self.model.predict(self.validation_data[0]), axis=1)
        val_targ = self.validation_data[1]
        _val_precision = precision_score(val_targ, val_predict, average=None, zero_division=0)
        _val_recall = recall_score(val_targ, val_predict, average=None, zero_division=0)
        _val_f1 = f1_score(val_targ, val_predict, average=None, zero_division=0)
        for i, (p, r, f) in enumerate(zip(_val_precision, _val_recall, _val_f1)):
            print(f" — Class {i} — val_precision: {p:.4f} — val_recall: {r:.4f} — val_f1: {f:.4f}")




Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_40 (Dense)    │ (None, 64)        │        896 │ input_layer_18[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1, 64)     │          0 │ dense_40[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 64)     │     66,368 │ lambda[0][0],     │
│ (MultiHeadAttentio… │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_49          │ (None, 1, 64)     │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_26 (Add)        │ (None, 1, 64)     │          0 │ lambda[0][0],     │
│                     │                   │            │ dropout_49[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 64)     │        128 │ add_26[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_41 (Dense)    │ (None, 1, 128)    │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_50          │ (None, 1, 128)    │          0 │ dense_41[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 1, 64)     │      8,256 │ dropout_50[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_27 (Add)        │ (None, 1, 64)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_42[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 64)     │        128 │ add_27[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 64)     │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_52          │ (None, 1, 64)     │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_28 (Add)        │ (None, 1, 64)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_52[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 64)     │        128 │ add_28[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 1, 128)    │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_53          │ (None, 1, 128)    │          0 │ dense_43[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 173,635 (678.26 KB)

 Trainable params: 173,635 (678.26 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Initialize the custom metric callback with validation data
metrics = Metrics(validation_data=(X_val, y_val))

# Train the model with the custom metrics callback and class weights
history = model.fit(X_train, y_train, epochs=50, batch_size=16, shuffle=True, validation_data=(X_val, y_val), verbose=1, callbacks=[metrics], class_weight=class_weights)

Epoch 1/50


/Users/vincent/mambaforge/envs/ml4qs/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (16, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


857/857 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 1.1631

/Users/vincent/mambaforge/envs/ml4qs/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Users/vincent/mambaforge/envs/ml4qs/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (32, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
 — Class 0 — val_precision: 0.0000 — val_recall: 0.0000 — val_f1: 0.0000
 — Class 1 — val_precision: 0.7077 — val_recall: 0.6747 — val_f1: 0.6908
 — Class 2 — val_precision: 0.1796 — val_recall: 0.3248 — val_f1: 0.2313
857/857 ━━━━━━━━━━━━━━━━━━━━ 77s 81ms/step - loss: 1.1631 - val_loss: 1.0548
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step ep -
 — Class 0 — val_precision: 0.1172 — val_recall: 0.7929 — val_f1: 0.2042
 — Class 1 — val_precision: 0.7241 — val_recall: 0.0175 — val_f1: 0.0341
 — Class 2 — val_precision: 0.1797 — val_recall: 0.1975 — val_f1: 0.1882
857/857 ━━━━━━━━━━━━━━━━━━━━ 65s 75ms/step - loss: 1.0966 - val_loss: 1.1738
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step ep 
 — Class 0 — val_precision: 0.0000 — val_recall: 0.0000 — val_f1: 0.0000
 — Class 1 — val_precision: 0.8140 — val_recall: 0.1165 — val_f1: 0.2038
 — Class 2 — val_precision: 0.1907 — val_recall: 0.9363 — val_f1: 0.3168
857/857 ━━━━━━━━━━━━━━━━━━━━ 64s 74ms/st

KeyboardInterrupt: 

In [4]:
# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=1)

# Calculate precision, recall, and F1 score for the test set for each class
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

print("Test Precision for each class:", precision)
print("Test Recall for each class:", recall)
print("Test F1 Score for each class:", f1)

108/108 [==============================] - 0s 2ms/step
Test Precision for each class: [0.08619092 0.85653409 0.16135881]
Test Recall for each class: [0.37804878 0.44014599 0.34311512]
Test F1 Score for each class: [0.14037736 0.58148505 0.21949458]
